In [ ]:
import numpy as np
from pathlib import Path
from keras.utils import to_categorical
from keras import Input, Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Rescaling, Flatten, Dense, Activation
from keras.optimizers import Adam
import os
import cv2
from sklearn.model_selection import train_test_split

    
def decoder(input_layer, filters, concat_layer):
    """up conv, concat, 2 convolutions"""
    conv_1 = Conv2D(filters, 2, activation="relu", padding="same")(UpSampling2D(size=(2, 2))(input_layer))
    concat = concatenate([concat_layer, conv_1], axis=-1)  # concatenate 2 layers
    conv_2 = Conv2D(filters, 3, activation="relu", padding="same")(concat)
    conv_3 = Conv2D(filters, 3, activation="relu", padding="same")(conv_2)

    return conv_3


def encode(input_layer, filters):
    """2 convolutions + 1 max pool (2x2)"""
    conv_1 = Conv2D(filters, 3, activation="relu", padding="same")(input_layer)
    conv_2 = Conv2D(filters, 3, activation="relu", padding="same")(conv_1)
    max_pool = MaxPooling2D(pool_size=(2, 2))(conv_2)

    return max_pool, conv_2


def Unet_model():

    inputs = Input((256, 256, 3))
    pool1, e1 = encode(inputs, 64)
    pool2, e2 = encode(pool1, 128)
    pool3, e3 = encode(pool2, 256)
    pool4, e4 = encode(pool3, 512)

    base = Conv2D(1024, 3, activation="relu", padding="same")(pool4)
    base = Conv2D(1024, 3, activation="relu", padding="same")(base)

    d4 = decoder(base, 512, e4)
    d3 = decoder(d4, 256, e3)
    d2 = decoder(d3, 128, e2)
    d1 = decoder(d2, 64, e1)

    output = Conv2D(2, 1, activation='softmax', name="output")(d1)

    return Model(inputs, output, name="U-Net")


data_path = r'C:\Users\User\Desktop\Master_Y1\Deep_learning\BrainTumorDetection-main\brain_tumor_dataset'
classes = ['no', 'yes']
img_size = 256

X = []
Y = []
for c in classes:
    path = os.path.join(data_path, c)
    class_num = classes.index(c)
    for img in os.listdir(path):
        img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        img_arr = cv2.resize(img_arr, (img_size, img_size))
        X.append(img_arr)
        Y.append(class_num)

X = np.array(X)
Y = np.array(Y)
Y = to_categorical(Y, num_classes=len(classes))

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


model = Sequential([Unet_model(), #original U-Net model
Flatten(), #flatten layer to 1d
Dense(2, activation='softmax') #add 1 fully connected layer to output node with 2 units
])
model.summary()

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

print("---------------Start fit (training)--------------------")
model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_val, Y_val))
model.save_weights(filepath="model_weights")
model.save(filepath=Path("brain_tumor_dataset"), overwrite=True)
